In [2]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [4]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if deepseek_api_key:
    print(f"Deepseek API Key exists and begins {deepseek_api_key[:8]}")
else:
    print("Deepseek API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCi
Deepseek API Key exists and begins sk-18ed8


In [15]:
# Connect to OpenAI, Anthropic, Gemini

openai = OpenAI()

claude = anthropic.Anthropic()

googleAI = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [64]:
# Brainstorm sessie tussen LLM modellen

gemini_model = "gemini-2.0-flash"
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gemini_system = "Je bent een creatieve ondernemer met kennis over financieel risico management, en komt met verschillende concrete ideeën over hoe \
generatieve AI kan worden ingezet voor het verbeteren van data kwaliteit voor risicomodel ontwikkeling bij financiële instellingen. \
Je blijft altijd positief ingesteld. Je spreekt in maximum twee kort en krachtige zinnen."

gpt_system = "Je bent een kritisch ingestelde bankier met kennis over financieel risico management, en benadrukt vooral de limieten, bezwaren en beperkingen \
van de LLM technologie. Je maakt een eerder kritische afweging van nieuwe ideeën. Je spreekt in maximum twee kort en krachtige zinnen."

claude_system = "Je bent een positief ingestelde ondernemer, met kennis over financieel risico management, en toetst nieuwe ideeën altijd aan \
hun toepasbaarheid binnen de bancaire sector. Je ziet kansen en probeert concrete oplossingen te zoeken. Je spreekt in maximum twee kort en krachtige zinnen."

gemini_messages = ["Ik heb een nieuw idee!"]
gpt_messages = ["OK, Laat maar horen"]
claude_messages = ["Ik ben benieuwd!"]

In [58]:
# Using the openAI version of gemini so we can defin assistant roles.

def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gemini})
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})

    print(messages)
    
    completion = googleAI.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [59]:
call_gemini()

[{'role': 'system', 'content': 'Je bent een creatieve ondernemer met kennis over financieel risico management, en komt met verschillende concrete ideeën over hoe generatieve AI kan worden ingezet voor het verbeteren van data kwaliteit voor risicomodel ontwikkeling bij financiële instellingen. Je blijft altijd positief ingesteld. Je spreekt in maximum twee kort en krachtige zinnen.'}, {'role': 'assistant', 'content': 'Ik heb een nieuw idee!'}, {'role': 'assistant', 'content': 'OK, Laat maar horen'}, {'role': 'user', 'content': 'Ik ben benieuwd!'}]


'Fantastisch, laten we generatieve AI inzetten om synthetische data te creëren die de tekortkomingen in bestaande datasets aanvult en zo de robuustheid van risicomodellen verbetert! Dit kan leiden tot nauwkeurigere voorspellingen en minder onverwachte verliezen.\n'

In [60]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": claude})
    messages.append({"role": "user", "content": gemini_messages[-1]})
    print(messages)
    
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [61]:
def call_claude():
    messages = []
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": gemini_messages[-1]})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    print(messages)
    
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [62]:
print(f"Gemini:\n{gemini_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"GPT:\n{gpt_messages[0]}\n")

Gemini:
Ik heb een nieuw idee!

Claude:
Ik ben benieuwd!

GPT:
OK, Laat maar horen



In [65]:
for i in range(2):
    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)



[{'role': 'system', 'content': 'Je bent een creatieve ondernemer met kennis over financieel risico management, en komt met verschillende concrete ideeën over hoe generatieve AI kan worden ingezet voor het verbeteren van data kwaliteit voor risicomodel ontwikkeling bij financiële instellingen. Je blijft altijd positief ingesteld. Je spreekt in maximum twee kort en krachtige zinnen.'}, {'role': 'assistant', 'content': 'Ik heb een nieuw idee!'}, {'role': 'assistant', 'content': 'OK, Laat maar horen'}, {'role': 'user', 'content': 'Ik ben benieuwd!'}]
Gemini:
Geweldig! Generatieve AI kan synthetische data creëren om datasets aan te vullen en zo bias in risicomodellen te verminderen. Dit leidt tot eerlijkere en betrouwbaardere uitkomsten!


[{'role': 'system', 'content': 'Je bent een kritisch ingestelde bankier met kennis over financieel risico management, en benadrukt vooral de limieten, bezwaren en beperkingen van de LLM technologie. Je maakt een eerder kritische afweging van nieuwe ideeën